# Predicting Budds text and reformatting its schema for CharaParser
## Workflow steps:

* Import text file by reading the lines
* Push the lines into the model pipeline
* add the classifications as XML nodes
* reformat to input schema

In [64]:
# Auto update of packages within the notebook
%load_ext autoreload
%autoreload 2

import os
import sys

# Import custom modelling code
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.predict_model import *
from src.features.build_length_features import locate_empty_strings

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
['/Users/jocelynpender/flora-prepper/notebooks/flora_commons_workflow', '/Users/jocelynpender/miniconda3/envs/flora-prepper/lib/python37.zip', '/Users/jocelynpender/miniconda3/envs/flora-prepper/lib/python3.7', '/Users/jocelynpender/miniconda3/envs/flora-prepper/lib/python3.7/lib-dynload', '', '/Users/jocelynpender/.local/lib/python3.7/site-packages', '/Users/jocelynpender/miniconda3/envs/flora-prepper/lib/python3.7/site-packages', '/Users/jocelynpender/miniconda3/envs/flora-prepper/lib/python3.7/site-packages/IPython/extensions', '/Users/jocelynpender/.ipython', '/Users/jocelynpender/flora-prepper/src', '/Users/jocelynpender/flora-prepper', '/Users/jocelynpender/flora-prepper', '/Users/jocelynpender/flora-prepper']


## Importing Budd's text

In [13]:
budds_file_name = "/Users/jocelynpender/flora-prepper-proj/data/external/texts/budds_testing.txt"
budds_lines = open(budds_file_name).read().splitlines()

In [ ]:
# if two key tags occur right after one another, merge the key tags into one key tag

## Import the vectorizer and create Budd's DTM

In [20]:
custom_vec = joblib.load("../../models/custom_vec") # load the DTM vectorizer
test_dtm = custom_vec.transform(budds_lines) # fit the DTM to the new data

## Import the classifier and generate predictions

In [22]:
# load the model and generate predictions
clf = joblib.load("../../models/classifier_model") # load the classifier
predicted = clf.predict(test_dtm) # predict the classification based on the DTM and the model
dtm_predictions_series = pd.Series(predicted)

## Reformat the results for easy viewing

In [37]:
budds_lines = pd.Series(budds_lines)
budds_results = pd.concat([budds_lines, dtm_predictions_series], axis=1)
budds_results = budds_results[locate_empty_strings(budds_results[0])] # Remove rows with empty strings as text
budds_results.to_csv(path_or_buf="budds_results_to_examine.csv")
budds_results

,0,1
0,Keys to main groups,key
1,"Key to Divisions, Subdivisions, and Classes",key
2,,morphology
3,"1. Plants without true flowers, reproducing",key
4,by spores,morphology
...,...,...
29680,3 ^073 0005Zb23 fl DATE DUE DEC,taxon_identification
29681,,morphology
29682,\ k ?(M»7 GAYLORO,taxon_identification
29683,PfllNKD IN USA.,habitat
